<a href="https://colab.research.google.com/github/xahram/Sci-Bert/blob/main/_Sci_Bert_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First step is to load the NIPS data that is uploaded in the Google Drive

In [ ]:
# Mount the google drive folder into the directory to access files

from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
# Load all necessary libraries

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import time 

nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Load the NIPS dataset from the drive

nips_papers_df = pd.read_csv('/gdrive/My Drive/Master_dataset/papers.csv')  
nips_papers_df.head()

nips_papers = nips_papers_df.infer_objects()

nips_papers.dtypes

nips_papers["year"] = pd.to_datetime(nips_papers["year"], format="%Y")
# nips_papers['year'] = nips_papers['year'].dt.year
nips_papers.sort_values(by='year')

print(nips_papers.dtypes)

max(nips_papers["year"])
min(nips_papers["year"])

nips_papers = nips_papers.sort_values(by = "year")


id                     int64
year          datetime64[ns]
title                 object
event_type            object
pdf_name              object
abstract              object
paper_text            object
dtype: object


In [ ]:
from os import close
# Slice Data Frame by 3 year interval


# print(len(nips_papers))

# Partition/Group Papers into df by the interval/freq of 3 years, closed = left to start combinbing from the 1987
nips_papers_3y_grouped = nips_papers.groupby(pd.Grouper(key='year', freq='3Y', sort=True, closed="left"))



# Save partitions in the Dictionary format with 10 intervals
nips_papers_partitions = {}
initial_partition_id = 0
for i, g  in nips_papers_3y_grouped:
    nips_papers_partitions[initial_partition_id] = g
    initial_partition_id = initial_partition_id + 1


print(nips_papers_partitions)
# nips_papers_three_year_partition[0].tail()


#for i, g in nips_papers.groupby(pd.Grouper(key=nips_papers["year"], freq='A')):
#     print(g)







{0:        id       year                                              title  \
0       1 1987-01-01  Self-Organization of Associative Database and ...   
328    13 1987-01-01   Temporal Patterns of Activity in Neural Networks   
6853   72 1987-01-01  Ensemble' Boltzmann Units have Collective Comp...   
6743   71 1987-01-01  Centric Models of the Orientation Map in Prima...   
6632   70 1987-01-01  On the Power of Neural Networks for Solving Ha...   
...   ...        ...                                                ...   
1650  250 1989-01-01                               Optimal Brain Damage   
1661  251 1989-01-01  A Self-organizing Associative Memory System fo...   
1672  252 1989-01-01  Can Simple Cells Learn Curves? A Hebbian Model...   
1683  253 1989-01-01  Subgrouping Reduces Complexity and Speeds Up L...   
1638  249 1989-01-01  Neural Network Analysis of Distributed Represe...   

     event_type                                           pdf_name  \
0           NaN  1-self-o

In [ ]:
# IMPORT word2phrase to create bigrams and unigrams
!git clone https://github.com/travisbrady/word2phrase.git

Cloning into 'word2phrase'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


# ALL TIME WINDOWS SCI-BERT

In [ ]:
# Convert List of Time Slice DF paper_text content to lists

start = time.time()

# Loop through every DF and convert paper_text to list and concatenate all the papers of one time slice 
## this will be a list like  ["All paper content string of first slice", "all paper content string of 2nd slice", ...] 

papers_contents_list = [" ".join(time_slice_df["paper_text"].tolist()) for time_slice_df in nips_papers_partitions.values()]

#### MEASURE THE EXECUTION TIME FOR RUNNING THE CONCATENATION CODE

end = time.time()
print(end - start)

# papers_contents_list
# len(papers_contents_list[0])

0.10246014595031738


In [ ]:
#  Join Paper titles for bigram and unigram extraction


papers_titles_list = [" ".join(time_slice_df["title"].tolist()) for time_slice_df in nips_papers_partitions.values()]



## Step 1 -  Pre Processing 

# Remove Stopwords 

In [ ]:
import re


# function to rmeove digits and numbers from papers 
def regex_remove_digits(papers_contents_list):      
    # Remove any digits for the corpus
    all_time_window_papers_content_list = [re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", time_slice_paper) 
                                                    for time_slice_paper in papers_contents_list] 
    # Remove words with length less than 3 

    # https://stackoverflow.com/questions/24332025/remove-words-of-length-less-than-4-from-string
    all_time_window_papers_content_list = [re.sub(r'\b\w{1,2}\b', '', time_slice_paper) 
                                          for time_slice_paper in all_time_window_papers_content_list]

    return all_time_window_papers_content_list






In [ ]:

# Custom Stopwords List for Scientific Literature 
from sklearn.feature_extraction import text

path_to_stop_words = '/gdrive/My Drive/Master_dataset/stopwords_10000_most_frequent_filtered.txt'

with open(path_to_stop_words, "r") as file1:
    FileasList = file1.readlines()


stopwords = [s.strip('\n') for s in FileasList]
print(len(stopwords))


scientific_literature_stopwords = text.ENGLISH_STOP_WORDS.union(stopwords)

len(scientific_literature_stopwords)


9954


9958

In [ ]:
# Get all paper content and titles for bigram and unigram generation
all_time_window_papers_content_list = regex_remove_digits(papers_contents_list)
all_time_window_papers_title_list = regex_remove_digits(papers_titles_list)


## Get Bag Of Candidate Keywords For All Time Windows

In [ ]:

all_time_window_papers_titles = " ".join(all_time_window_papers_title_list)
all_time_window_papers_titles = all_time_window_papers_titles.replace(',', '').replace(';', '').replace(":", "")

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from nltk.corpus import stopwords

#About 900 stopwords
stop_words = list(stopwords.words('english')) #About 150 stopwords
stop_words.extend(scientific_literature_stopwords)



token = nltk.word_tokenize(all_time_window_papers_titles)
output = [lemmatizer.lemmatize(w) for w in token if not w in stop_words]

unigrams = ngrams(output,1)
bigrams = ngrams(output,2)



# candidate_keywords = [ for n in ngrams]
ngrams = Counter(bigrams).most_common()
unigrams = Counter(unigrams).most_common()


# unigrams = [ (n[0][0], n[1]) for n in unigrams]
unigrams

[(('Learning',), 1447),
 (('Neural',), 565),
 (('Networks',), 548),
 (('Models',), 407),
 (('Model',), 331),
 (('Bayesian',), 287),
 (('Using',), 260),
 (('Network',), 240),
 (('The',), 225),
 (('Inference',), 221),
 (('Data',), 221),
 (('Analysis',), 215),
 (('Gaussian',), 202),
 (('Classification',), 194),
 (('Optimization',), 192),
 (('Sparse',), 186),
 (('Stochastic',), 181),
 (('Deep',), 170),
 (('Estimation',), 168),
 (('Recognition',), 163),
 (('Efficient',), 163),
 (('Clustering',), 162),
 (('Algorithms',), 160),
 (('Kernel',), 155),
 (('Reinforcement',), 151),
 (('Algorithm',), 144),
 (('Online',), 144),
 (('Adaptive',), 143),
 (('Linear',), 138),
 (('Regression',), 138),
 (('Optimal',), 136),
 (('Fast',), 136),
 (('Markov',), 133),
 (('Information',), 132),
 (('Visual',), 131),
 (('Image',), 128),
 (('Approach',), 125),
 (('Probabilistic',), 121),
 (('?',), 121),
 (('Processes',), 120),
 (('Feature',), 114),
 (('Gradient',), 113),
 (('Variational',), 113),
 (('Prediction',), 

In [ ]:
ngrams.extend(unigrams)
# ngrams.extend([])

len(ngrams)

40045

In [ ]:


# Sort the bag of words
ngrams = sorted(ngrams, key=lambda item: item[1], reverse=True)



In [ ]:
candidate_keywords = [( " ".join(n[0]).lower() , n[1] ) for n in ngrams]
candidate_keywords = candidate_keywords[:500]



###################################################

In [ ]:
candidate_keywords

[('learning', 1447),
 ('neural', 565),
 ('networks', 548),
 ('models', 407),
 ('model', 331),
 ('bayesian', 287),
 ('using', 260),
 ('neural networks', 247),
 ('network', 240),
 ('the', 225),
 ('inference', 221),
 ('data', 221),
 ('analysis', 215),
 ('gaussian', 202),
 ('classification', 194),
 ('optimization', 192),
 ('sparse', 186),
 ('stochastic', 181),
 ('deep', 170),
 ('estimation', 168),
 ('recognition', 163),
 ('efficient', 163),
 ('clustering', 162),
 ('algorithms', 160),
 ('kernel', 155),
 ('reinforcement', 151),
 ('reinforcement learning', 148),
 ('algorithm', 144),
 ('online', 144),
 ('adaptive', 143),
 ('linear', 138),
 ('regression', 138),
 ('optimal', 136),
 ('fast', 136),
 ('neural network', 135),
 ('markov', 133),
 ('information', 132),
 ('visual', 131),
 ('image', 128),
 ('approach', 125),
 ('probabilistic', 121),
 ('?', 121),
 ('processes', 120),
 ('feature', 114),
 ('gradient', 113),
 ('variational', 113),
 ('prediction', 111),
 ('modeling', 109),
 ('process', 109),


## Monitoring Keywords

In [ ]:
vec2dynamics_keywords = ['neural network', 'reinforcement learning', 'active learning', 'monte carlo', 'deep learning',
                          'machine learning', 'supervised learning', 'time series', 'artificial neural',
                         'gaussian process', 'active learning', 'gradient descent', 'hidden markov',
                         'nearest neighbor', 'dynamical system', 'dimensionality reduction',
                         'unsupervised learning', 'graphical model', 'dynamic programming', 'component analysis', 'neural'
                         ,'supervised','learning', 'model', 'reinforcement', 'markov', 'gradient', 'markov', 'neighbor','gaussian']

In [ ]:
# keywords_np = np.array(candidate_keywords)

candidate_keywords_ = np.array([keyword[0] for keyword in candidate_keywords])

candidate_keywords_

array(['learning', 'neural', 'networks', 'models', 'model', 'bayesian',
       'using', 'neural networks', 'network', 'the', 'inference', 'data',
       'analysis', 'gaussian', 'classification', 'optimization', 'sparse',
       'stochastic', 'deep', 'estimation', 'recognition', 'efficient',
       'clustering', 'algorithms', 'kernel', 'reinforcement',
       'reinforcement learning', 'algorithm', 'online', 'adaptive',
       'linear', 'regression', 'optimal', 'fast', 'neural network',
       'markov', 'information', 'visual', 'image', 'approach',
       'probabilistic', '?', 'processes', 'feature', 'gradient',
       'variational', 'prediction', 'modeling', 'process', 'methods',
       'robust', 'dynamic', 'recurrent', 'hierarchical', 'detection',
       'training', 'random', 'time', 'structured', 'control', 'functions',
       'selection', 'application', 'bounds', 'sampling',
       'representations', 'local', 'latent', 'structure', 'unsupervised',
       'model', 'machine', 'search',

In [ ]:
list(set(vec2dynamics_keywords) - set(candidate_keywords_))

['graphical model',
 'dynamical system',
 'artificial neural',
 'nearest neighbor']

In [ ]:
monitering_keywords = list(set(vec2dynamics_keywords).intersection(set(candidate_keywords_)))
monitering_keywords = monitering_keywords[:20]

In [ ]:

# candidate_keywords = candidate_keywords_

len(monitering_keywords)

20

## Step 2.1 - Get Vocabulary list

In [ ]:
papers_contents_list = [" ".join(time_slice_df["paper_text"].tolist()).lower() for time_slice_df in nips_papers_partitions.values()]


In [ ]:
import re


# function to rmeove digits and numbers from papers 
def v_preprocess(papers_contents_list):      
    # Remove any digits for the corpus
    all_time_window_papers_content_list = list( (re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", time_slice_paper) 
                                                    for time_slice_paper in papers_contents_list) )
    # Remove words with length less than 3 

    # https://stackoverflow.com/questions/24332025/remove-words-of-length-less-than-4-from-string
    all_time_window_papers_content_list = list( (re.sub(r'\b\w{1,2}\b', '', time_slice_paper) 
                                          for time_slice_paper in all_time_window_papers_content_list) )


    # remove words with number in it 
    # https://stackoverflow.com/questions/18082130/python-regex-to-remove-all-words-which-contains-number

    all_time_window_papers_content_list = list( (re.sub(r'\w*\d\w*', '', time_slice_paper) 
                                          for time_slice_paper in all_time_window_papers_content_list) )
    return all_time_window_papers_content_list






In [ ]:
len(papers_contents_list[9])

46835872

In [ ]:
papers_contents_list = v_preprocess(papers_contents_list)

In [ ]:
len(papers_contents_list[9])

41875126

In [ ]:
# For optimization purpose
import nltk
def vocabulary(C):

    v = list(set( (lemmatizer.lemmatize(w) for w in nltk.word_tokenize(C) if w not in stop_words) ))
    return v

In [ ]:
# import time

# start = time.time()

# v10 = vocabulary(papers_contents_list[9])

# end = time.time()
# print(end - start)

In [ ]:
v1 = vocabulary(papers_contents_list[0][:100000])

In [ ]:
v2 = vocabulary(papers_contents_list[1][:100000]) 
v3 = vocabulary(papers_contents_list[2][:100000]) 
v4 = vocabulary(papers_contents_list[3][:100000]) 
v5 = vocabulary(papers_contents_list[4][:100000]) 
v6 = vocabulary(papers_contents_list[5][:100000]) 


In [ ]:
v_map = {

    "v1":v1,
    "v2":v2,
    "v3":v3,
    "v4":v4,
    "v5":v5,
    "v6":v6
}

import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_2.pickle', 'wb+') as f:
     pickle.dump(v_map, f)

In [ ]:
v7 = vocabulary(papers_contents_list[6][:100000]) 
v8 = vocabulary(papers_contents_list[7][:100000]) 


v_map2 = {

    "v7":v7,
    "v8":v8,
}


import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_2.pickle', 'ab+') as f:
     pickle.dump(v_map2, f)

In [ ]:

v9 = vocabulary(papers_contents_list[8][:100000]) 
v10 = vocabulary( papers_contents_list[9][:100000]) 


v_map3 = {
    
    "v9":v9,
    "v10":v10
}



import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_2.pickle', 'ab+') as f:
     pickle.dump(v_map3, f)

In [ ]:
import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_2.pickle', 'rb+') as f:
     v_map_1 = pickle.load(f)
     v_map_2 = pickle.load(f)
     v_map_3 = pickle.load(f)



In [ ]:
v_map = {
    **v_map_1,
    **v_map_2,
    **v_map_3
}


v1 = v_map["v1"]
v2 = v_map["v2"]
v3 = v_map["v3"]
v4 = v_map["v4"]
v5 = v_map["v5"]
v6 = v_map["v6"]
v7 = v_map["v7"]
v8 = v_map["v8"]
v9 = v_map["v9"]
v10 = v_map["v10"]




In [ ]:
print(f"Length of \n V1={len(v1)} \n V2={len(v2)} \n V3={len(v3)} \n V4={len(v4)} \n V5={len(v5)} \n V6={len(v6)} \n V7={len(v7)} \n V8={len(v8)} \n V9={len(v9)} V10={len(v10)}")

Length of 
 V1=1276 
 V2=1200 
 V3=1441 
 V4=1456 
 V5=1244 
 V6=1103 
 V7=1216 
 V8=1205 
 V9=1179 V10=1248


### Step 3 - BERT EMBEDDING GENERATE

In [ ]:
title_ngram_candidate_keywords_time_slices_sorted_ = [ngram[0] for ngram in candidate_keywords]
# t = "\n".join(title_ngram_candidate_keywords_time_slices_sorted_)
t = "\n".join(monitering_keywords)

t

'markov\ndeep learning\nsupervised\ngradient descent\nsupervised learning\nneural network\ndimensionality reduction\ncomponent analysis\ntime series\nneural\nneighbor\nmonte carlo\nreinforcement learning\nhidden markov\nmodel\nreinforcement\ngradient\nlearning\nmachine learning\ngaussian'

In [ ]:
!pip install pytorch-pretrained-bert
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 29.2 MB/s 
     |████████████████████████████████| 132 kB 63.8 MB/s 
     |████████████████████████████████| 9.1 MB 49.9 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 140 kB 68.7 MB/s 
     |████████████████████████████████| 127 kB 70.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 24.5 MB/s 


In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
from collections import OrderedDict
import unidecode
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
import os
import matplotlib.pyplot as plt
# % matplotlib inline

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import torch
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla T4'

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#Use the pre-trained Base BERT model 
model = BertModel.from_pretrained('bert-base-uncased')
model.cuda()
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

100%|██████████| 407873900/407873900 [00:34<00:00, 11698618.93B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [ ]:
from collections import OrderedDict
class Data():

    def __getitem__(self, content=None):
         if content!=None:
             self.doc = "".join(content)
         return self.doc
     
    def _preprocess(self,targets,corpus):
        self.index=[]
        self.t_index=OrderedDict()
        for target in targets:
            
            for _,item in enumerate(corpus):
                # if target in item:
                  if item.lower().find(target) != -1:
                # if bool(re.search(target, item)):

                  # Check if sentence has multiple occurances of the target term
                      count_target=item.count(target)
                  #   Avoiding the sentences with multiple occurrences of the target term for the time being###
                      if count_target==1:
                        if target not in self.t_index.keys():
                            self.t_index[target]=[_]
                        else:
                            self.t_index[target].append(_)
                        self.index.append(_)
        return self.index,self.t_index

In [ ]:
'''
LOAD & EXTRACT DATA
'''
import os



p1 = nips_papers_partitions[0]["paper_text"].tolist()
p2 = nips_papers_partitions[1]["paper_text"].tolist()
p3 = nips_papers_partitions[2]["paper_text"].tolist()
p4 = nips_papers_partitions[3]["paper_text"].tolist()
p5 = nips_papers_partitions[4]["paper_text"].tolist()
p6 = nips_papers_partitions[5]["paper_text"].tolist()
p7 = nips_papers_partitions[6]["paper_text"].tolist()
p8 = nips_papers_partitions[7]["paper_text"].tolist()
p9 = nips_papers_partitions[8]["paper_text"].tolist()
p10 = nips_papers_partitions[9]["paper_text"].tolist()





t = t
datasets = Data() 

# doc1 =  ["Sentence1", "Sentence2".....]
doc1=datasets.__getitem__(p1).split('\n')   
doc2=datasets.__getitem__(p2).split('\n')
doc3=datasets.__getitem__(p3).split('\n')
doc4=datasets.__getitem__(p4).split('\n')
doc5=datasets.__getitem__(p5).split('\n')
doc6=datasets.__getitem__(p6).split('\n')
doc7=datasets.__getitem__(p7).split('\n')
doc8=datasets.__getitem__(p8).split('\n')
doc9=datasets.__getitem__(p9).split('\n')
doc10=datasets.__getitem__(p10).split('\n')


t1=datasets.__getitem__(t).split('\n')
target_act=[x for x in t1 if len(x)>1]
t1=[x.lower() for x in t1 if len(x)>1]

index1=datasets._preprocess(t1,doc1)
index2=datasets._preprocess(t1,doc2)
index3=datasets._preprocess(t1,doc3)
index4=datasets._preprocess(t1,doc4)
index5=datasets._preprocess(t1,doc5)
index6=datasets._preprocess(t1,doc6)
index7=datasets._preprocess(t1,doc7)
index8=datasets._preprocess(t1,doc8)
index9=datasets._preprocess(t1,doc9)
index10=datasets._preprocess(t1,doc10)


index_t1=index1[1]
index_t2=index2[1]
index_t3=index3[1]
index_t4=index4[1]
index_t5=index5[1]
index_t6=index6[1]
index_t7=index7[1]
index_t8=index8[1]
index_t9=index9[1]
index_t10=index10[1]

print('The target words are:',t1)
target_words=t1

print('The index_t1 are ', index_t1)
print('The index_t2 are ', index_t2)


#conversions
target_uni=[unidecode.unidecode(m) for m in t1]
target_toks=[]
# print(target_uni)
for k in t1:
  target_toks.append(tokenizer.tokenize(k))
print('converted target toks',target_toks)

The target words are: ['markov', 'deep learning', 'supervised', 'gradient descent', 'supervised learning', 'neural network', 'dimensionality reduction', 'component analysis', 'time series', 'neural', 'neighbor', 'monte carlo', 'reinforcement learning', 'hidden markov', 'model', 'reinforcement', 'gradient', 'learning', 'machine learning', 'gaussian']
The index_t1 are  OrderedDict([('markov', [2697, 2814, 3009, 110171]), ('supervised', [628, 1672, 19713, 19714, 19734, 19736, 19796, 19814, 19819, 20208, 20218, 20220, 20353, 20629, 26312, 26333, 28033, 28055, 28056, 28058, 28533, 28627, 28633, 28643, 28654, 29969, 31446, 31919, 32017, 32018, 32038, 32129, 32138, 32146, 32147, 32702, 38083, 38254, 51732, 51734, 51768, 54331, 54422, 55325, 58202, 58644, 59027, 59042, 59043, 59076, 59605, 60074, 60078, 60543, 60545, 60546, 60553, 61176, 61183, 61260, 61293, 61346, 61360, 61361, 61366, 61373, 61411, 61421, 61457, 61479, 61502, 61504, 64783, 64785, 65291, 65294, 70013, 70014, 71120, 71146, 7158

In [ ]:
len(t1)
print(len(index_t1))
print(len(index_t2))
# target_toks

len(list(index_t1.values())[1])


18
19


150

In [ ]:
def _pre_bert(doc,index,t):
  
  # index =  index_t1 -> { target_w1: index, target_w2: index2, target_w1 : index5 } -  index = Sentence index in which target word appears
  s=["Not Found"]  
  
  if t in index.keys():
      s=[doc[ind] for ind in index[t]]

  print('len of sentences',len(s))
  l=len(s)
  marked_text = ["[CLS] " + text + " [SEP]" for text in s]
  tokenized_text = [tokenizer.tokenize(m) for m in marked_text]
  
  tokenized_text=[x[:512] if len(x)>512 else x for x in tokenized_text]
  indexed_tokens = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
  segments_ids = [[1] * len(x) for x in tokenized_text]
  return s,marked_text,tokenized_text,indexed_tokens,segments_ids,l


def _bert_features(tokens_tensor, segments_tensors,tokenized_text):
  # print(len(tokens_tensor[0]))
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor.to(device), segments_tensors.to(device))
  # print ("Number of layers:", len(encoded_layers))
  layer_i = 0

  # # print ("Number of batches:", len(encoded_layers[layer_i]))
  batch_i = 0

  # print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
  token_i = 0

  # print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))
  # Convert the hidden state embeddings into single token vectors

  # Holds the list of 12 layer embeddings for each token
  # Will have the shape: [# tokens, # layers, # features]
  token_embeddings = [] 

  # For each token in the sentence...
  # tokenized_text=[x for x in tokenized_text if x not in ['_', 'n', '##n','v', '##b']]
  for token_i in range(len(tokenized_text)):
    
    # Holds 12 layers of hidden states for each token 
    hidden_layers = [] 
    
    # For each of the 12 layers...
    for layer_i in range(len(encoded_layers)):
      
      # Lookup the vector for `token_i` in `layer_i`
      vec = encoded_layers[layer_i][batch_i][token_i]
      
      hidden_layers.append(vec)
      
    token_embeddings.append(hidden_layers)

  # Sanity check the dimensions:
  # print ("Number of tokens in sequence:", len(token_embeddings))
  # print ("Number of layers per token:", len(token_embeddings[0]))
  return token_embeddings
# s,marked_text,tokenized_text,indexed_tokens,segments_ids
def _get_embeddings(pre,tg):
  m_embed_full=[]
  # print('len(pre[0])',len(pre[0]))
  # print(tg)
  for _,item in enumerate(pre[0]):
    # Convert inputs to PyTorch tensors
    # print(item)
    token_list=pre[2][_]
    
    tokens_tensor = torch.tensor([pre[3][_]])
    segments_tensors = torch.tensor([pre[4][_]])
    # Predict hidden states features for each layer
    token_embeddings=_bert_features(tokens_tensor, segments_tensors,pre[2][_])
    concatenated_last_4_layers = [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in token_embeddings] # [number_of_tokens, 3072]

    summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]
    
    #consider the tokenized target
  
    indxs=[]
    # print(token_list)
    for tok in tg:
      '''
      remove -1,-2,-3
      '''
      if tok in token_list:
        if tok not in ['_', 'n', '##n','v', '##b']:
          indxs.append(token_list.index(tok))

    # print('indxs',indxs)
    if len(indxs)==1:
      #bert_embed=concatenated_last_4_layers [indxs[0]]
      bert_embed=summed_last_4_layers [indxs[0]]

      m_embed_full.append(bert_embed)
    elif len(indxs)>1:
      b_emb=[]
      for ind in indxs:
        #b_emb.append(concatenated_last_4_layers[ind])
        b_emb.append(summed_last_4_layers[ind])
        
      bert_embed= torch.sum(torch.stack(b_emb), 0)
      m_embed_full.append(bert_embed)
    # indx=token_list.index(tg.lower())
    # indx = [i for (i, elem) in enumerate(pre[2][_]) if t in elem]
    # print('indx',indx)
    # print(pre[1][_],indx)

    # if len(indx)>0:
    # bert_embed=concatenated_last_4_layers[indx[0]]
    
    # cosine_similarity(summed_last_4_layers[10].reshape(1,-1), summed_last_4_layers[19].reshape(1,-1))[0][0]
    
    
  return  m_embed_full, summed_last_4_layers
# For a particular target word,do clustering and find if there is a sense change


In [ ]:
# Embedding for all time windows

sents_all=[]

# Holds all bert embeddings 
# [] -> [[embedding1], [embedding2],...]
X=[]

def embeddings_extract(target_words,target_toks,doc1,index_t1):
  t=target_words
  X_C1=[]
  lens1=[]
  for k,t in enumerate(target_words) :
    berts=[]
    sents=[]
    print('The target word is',t)    
    
    #get the sentences from corpus c1 and c2 for the specific target word 't'
    
    # This will generate tokenized sentences, tokens for the specific word. Or sentences containing specific word
    pre1=_pre_bert(doc1,index_t1,t)

    # lens1.append(pre1[-1])
    # lens2.append(pre2[-1])
    # print(pre1)
    
    sents.extend(pre1[0])
    #aggregate all the embeddings
    # s,marked_text,tokenized_text,indexed_tokens,segments_ids

    '''
    Get the embeddings of the targets from corpus 1 and 2
    '''
    _ , b1=_get_embeddings(pre1,target_toks[k])
    print('len of t1',len(b1))
    
    '''
    store the lenghts of no. of sentences extracted for each target word for each corpus
    '''
    lens1.append(len(b1))
    
    berts.extend(b1)
    print('len of each target word extractions is',len(berts))
    X.append(berts)

    # ______________ Placeholder to flatten the tensors into 1-D tensor for the 
    #           respective sentence tensors of specific keyword _______________ (b1)

    X_C1.append(b1)# the embeddings for C1
    sents_all.append(sents)
  return X,X_C1,lens1,sents_all



In [ ]:
#### BERT FOR Vocabulary

def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors









def extract_v_embeddings(v):
    target_word_embeddings = []
    for text in v:
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
        list_token_embeddings = _bert_features(tokens_tensor, segments_tensors, tokenized_text)
        summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in list_token_embeddings] # [number_of_tokens, 768]
        bert_embed= torch.sum(torch.stack(summed_last_4_layers), 0)
        # # Find the position 'bank' in list of tokens
        # word_index = tokenized_text.index('bank')
        # # Get the embedding for bank
        # word_embedding = list_token_embeddings[word_index]

        # target_word_embeddings.append(word_embedding)
        target_word_embeddings.append(bert_embed)
        
    return  target_word_embeddings



In [ ]:
import time

start = time.time()
v1_emb = extract_v_embeddings(v1)

end = time.time()
print(f'Time {end - start} \n')
len(v1_emb)


In [ ]:
v1_emb = extract_v_embeddings(v1)
v2_emb = extract_v_embeddings(v2)
v3_emb = extract_v_embeddings(v3)
v4_emb = extract_v_embeddings(v4)
v5_emb = extract_v_embeddings(v5)



In [ ]:
v_map1 = {

    "v1_emb":v1_emb,
    "v2_emb":v2_emb,
    "v3_emb":v3_emb,
    "v4_emb":v4_emb,
    "v5_emb":v5_emb
}

import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_emb2.pickle', 'wb+') as f:
     pickle.dump(v_map1, f)

In [ ]:

v6_emb = extract_v_embeddings(v6)
v7_emb = extract_v_embeddings(v7)
v8_emb = extract_v_embeddings(v8)

v_map2 = {

    "v6_emb":v6_emb,
    "v7_emb":v7_emb,
    "v8_emb":v8_emb
}

import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_emb2.pickle', 'ab+') as f:
     pickle.dump(v_map2, f)

In [ ]:

v9_emb = extract_v_embeddings(v9)
v10_emb = extract_v_embeddings(v10)


v_map3 = {

    "v9_emb":v9_emb,
    "v10_emb":v10_emb
}

import pickle
with open('/gdrive/My Drive/Master_dataset/vocabulary_emb2.pickle', 'ab+') as f:
     pickle.dump(v_map3, f)

In [ ]:
# import time
# start_time = time.time()

# embed_full,embed_C1,len_c1,sents=embeddings_extract(target_words,target_toks,doc1,index_t1)

# lens=[len_c1]
# # lens.append(len_c2)
# print('saved')
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print("")

In [ ]:
# BERT Embeddings

embed_full,embed_C1,len_c1,sents=embeddings_extract(target_words,target_toks,doc1,index_t1)
embed_full,embed_C2,len_c2,sents=embeddings_extract(target_words,target_toks,doc2,index_t2)
embed_full,embed_C3,len_c3,sents=embeddings_extract(target_words,target_toks,doc3,index_t3)
embed_full,embed_C4,len_c4,sents=embeddings_extract(target_words,target_toks,doc4,index_t4)
embed_full,embed_C5,len_c5,sents=embeddings_extract(target_words,target_toks,doc5,index_t5)


The target word is markov
len of sentences 4
len of t1 23
len of each target word extractions is 23
The target word is deep learning
len of sentences 1
len of t1 4
len of each target word extractions is 4
The target word is supervised
len of sentences 150
len of t1 22
len of each target word extractions is 22
The target word is gradient descent
len of sentences 106
len of t1 12
len of each target word extractions is 12
The target word is supervised learning
len of sentences 75
len of t1 22
len of each target word extractions is 22
The target word is neural network
len of sentences 969
len of t1 16
len of each target word extractions is 16
The target word is dimensionality reduction
len of sentences 9
len of t1 16
len of each target word extractions is 16
The target word is component analysis
len of sentences 7
len of t1 16
len of each target word extractions is 16
The target word is time series
len of sentences 33
len of t1 20
len of each target word extractions is 20
The target word i

In [ ]:
saved_map3 = {

    "embed_C1":embed_C1,
    "embed_C2":embed_C2,
    "embed_C3":embed_C3,
    "embed_C4":embed_C4,
    "embed_C5":embed_C5
}

import pickle
with open('/gdrive/My Drive/Master_dataset/bert_embeddings_4.pickle', 'wb+') as f:
     pickle.dump(saved_map3, f)

In [ ]:
print("")

In [ ]:
embed_full,embed_C6,len_c6,sents=embeddings_extract(target_words,target_toks,doc6,index_t6)
embed_full,embed_C7,len_c7,sents=embeddings_extract(target_words,target_toks,doc7,index_t7)
embed_full,embed_C8,len_c8,sents=embeddings_extract(target_words,target_toks,doc8,index_t8)





saved_map4 =  {
    "embed_C6":embed_C6,
    "embed_C7":embed_C7,
    "embed_C8":embed_C8
    }

import pickle
with open('/gdrive/My Drive/Master_dataset/bert_embeddings_4.pickle', 'ab+') as f:
     pickle.dump(saved_map4, f)

The target word is markov
len of sentences 54
len of t1 32
len of each target word extractions is 32
The target word is deep learning
len of sentences 1
len of t1 4
len of each target word extractions is 4
The target word is supervised
len of sentences 737
len of t1 27
len of each target word extractions is 27
The target word is gradient descent
len of sentences 115
len of t1 26
len of each target word extractions is 26
The target word is supervised learning
len of sentences 323
len of t1 28
len of each target word extractions is 28
The target word is neural network
len of sentences 335
len of t1 25
len of each target word extractions is 25
The target word is dimensionality reduction
len of sentences 201
len of t1 37
len of each target word extractions is 37
The target word is component analysis
len of sentences 132
len of t1 16
len of each target word extractions is 16
The target word is time series
len of sentences 100
len of t1 33
len of each target word extractions is 33
The target

In [ ]:
embed_full,embed_C9,len_c9,sents=embeddings_extract(target_words,target_toks,doc9,index_t9)
embed_full,embed_C10,len_c10,sents=embeddings_extract(target_words,target_toks,doc10,index_t10)



saved_map5 = {
    "embed_C9":embed_C9,
    "embed_C10":embed_C10,
    "sents":sents,
    "embed_full":embed_full
}
    

import pickle
with open('/gdrive/My Drive/Master_dataset/bert_embeddings_4.pickle', 'ab+') as f:
     pickle.dump(saved_map5, f)

The target word is markov
len of sentences 136
len of t1 15
len of each target word extractions is 15
The target word is deep learning
len of sentences 95
len of t1 17
len of each target word extractions is 17
The target word is supervised
len of sentences 1583
len of t1 39
len of each target word extractions is 39
The target word is gradient descent
len of sentences 553
len of t1 19
len of each target word extractions is 19
The target word is supervised learning
len of sentences 565
len of t1 40
len of each target word extractions is 40
The target word is neural network
len of sentences 710
len of t1 21
len of each target word extractions is 21
The target word is dimensionality reduction
len of sentences 221
len of t1 29
len of each target word extractions is 29
The target word is component analysis
len of sentences 288
len of t1 34
len of each target word extractions is 34
The target word is time series
len of sentences 379
len of t1 33
len of each target word extractions is 33
The t

## Apply K-NN with Cosine Similarity 

In [ ]:
import pickle
import numpy as np
import pandas as pd
import nltk
import torch


In [ ]:
with open('/gdrive/My Drive/Master_dataset/bert_embeddings_4.pickle', 'rb+') as f:
  saved_map3 = pickle.load(f)
  saved_map4 = pickle.load(f)
  saved_map5 = pickle.load(f)


In [ ]:
saved_map = {
    **saved_map3,
    **saved_map4,
    **saved_map5
}

In [ ]:
type(saved_map["embed_C7"][0][0])

torch.Tensor

In [ ]:
embed_C1 = saved_map["embed_C1"]
embed_C2 = saved_map["embed_C2"]
embed_C3 = saved_map["embed_C3"]
embed_C4 = saved_map["embed_C4"]
embed_C5 = saved_map["embed_C5"]
embed_C6 = saved_map["embed_C6"]
embed_C7 = saved_map["embed_C7"]
embed_C8 = saved_map["embed_C8"]
embed_C9 = saved_map["embed_C9"]
embed_C10 = saved_map["embed_C10"]

In [ ]:
# Apply K-NN to calculate the Nearest neighbor of keywords

def convert_tensors_tolist(mapping):
  for i, word_sentences in enumerate(mapping):
        # Use below line whe converting tensors to numpy array
        X1=np.array([np.array(x.to('cpu')) for x in word_sentences])

        # X1=np.array([np.array(x) for x in word_sentences])
        X1=X1.sum(axis=0).tolist()
        mapping[i] = X1
  return mapping



def convert_v_tolist(v):

  for i, word in enumerate(v):
        # Use below line whe converting tensors to numpy array
        w = np.array(word.to('cpu')).tolist()
        v[i] = w

  return v

In [ ]:
##### FLATTEN tensors of sentences of respective words to 1-D

embed_C1_ = convert_tensors_tolist(embed_C1)
embed_C2_ = convert_tensors_tolist(embed_C2)
embed_C3_ = convert_tensors_tolist(embed_C3)
embed_C4_ = convert_tensors_tolist(embed_C4)
embed_C5_ = convert_tensors_tolist(embed_C5)
embed_C6_ = convert_tensors_tolist(embed_C6)
embed_C7_ = convert_tensors_tolist(embed_C7)
embed_C8_ = convert_tensors_tolist(embed_C8)
embed_C9_ = convert_tensors_tolist(embed_C9)
embed_C10_ = convert_tensors_tolist(embed_C10)



In [ ]:
v1_emb_ = convert_v_tolist(v1_emb)
v2_emb_ = convert_v_tolist(v2_emb)
v3_emb_ = convert_v_tolist(v3_emb)
v4_emb_ = convert_v_tolist(v4_emb)
v5_emb_ = convert_v_tolist(v5_emb)
v6_emb_ = convert_v_tolist(v6_emb)
v7_emb_ = convert_v_tolist(v7_emb)
v8_emb_ = convert_v_tolist(v8_emb)
v9_emb_ = convert_v_tolist(v9_emb)
v10_emb_ = convert_v_tolist(v10_emb)


In [ ]:
len(embed_C2_[0])

768

In [ ]:
from sklearn.neighbors import NearestNeighbors

def _nn_(V, X):
  model = NearestNeighbors(n_neighbors=10,
                          metric='cosine',
                          algorithm='brute',
                          n_jobs=-1)

  n_n = model.fit(V)  
  distance, indeces = model.kneighbors(X)


  return indeces


In [ ]:
# indeces_1 = nn_(embed_C1_)

indeces1 = _nn_(v1_emb_ , embed_C1_)
indeces2 = _nn_(v2_emb_ , embed_C2_)
indeces3 = _nn_(v3_emb_ , embed_C3_)
indeces4 = _nn_(v4_emb_ , embed_C4_)
indeces5 = _nn_(v5_emb_ , embed_C5_)
indeces6 = _nn_(v6_emb_ , embed_C6_)
indeces7 = _nn_(v7_emb_ , embed_C7_)
indeces8 = _nn_(v8_emb_ , embed_C8_)
indeces9 = _nn_(v9_emb_ , embed_C9_)
indeces10 = _nn_(v10_emb_ , embed_C10_)





In [ ]:
len(v1_emb_)

1276

In [ ]:
indeces1[0]

array([ 409,  741,   53, 1081, 1221,  800,  494,  822,  689,  225])

## GET NEAREST NEIGHBORS

In [ ]:


def get_nearest_keywords_(indeces, v,keywords_np=monitering_keywords):
  tup_nearest_neighbor = []
  v = np.array(v)
  for index, candidate_keyword in enumerate(keywords_np):
      # Take the current index of the keyword and get the list of 10 nearest index from KNN algorithm
      nearest_neighbors_indeces_of_current_keyword = indeces[index]

      # Filter the keyword list using the list of indeces obtained in previous step
      nearest_keywords = v[nearest_neighbors_indeces_of_current_keyword]

      # Create tuple with first element as the keyword for current iteration and 2nd element as list of its nearest neighbors
      tup_nearest_neighbor.append({candidate_keyword : set(nearest_keywords)})


  return tup_nearest_neighbor

In [ ]:
nn_1 = get_nearest_keywords_(indeces1, v1)
nn_2 = get_nearest_keywords_(indeces2, v2)
nn_3 = get_nearest_keywords_(indeces3, v3)
nn_4 = get_nearest_keywords_(indeces4, v4)
nn_5 = get_nearest_keywords_(indeces5, v5)
nn_6 = get_nearest_keywords_(indeces6, v6)
nn_7 = get_nearest_keywords_(indeces7, v7)
nn_8 = get_nearest_keywords_(indeces8, v8)
nn_9 = get_nearest_keywords_(indeces9, v9)
nn_10 = get_nearest_keywords_(indeces10, v10)


In [ ]:
nn_2

In [ ]:
def get_target_nn(nn, monitering_keywords = monitering_keywords):
    return [n for n in nn for key in list(n.keys()) if key in monitering_keywords]
    

In [ ]:
nn_2_ = get_target_nn(nn_2)
nn_3_ = get_target_nn(nn_3)
nn_4_ = get_target_nn(nn_4)
nn_5_ = get_target_nn(nn_5)
nn_6_ = get_target_nn(nn_6)
nn_7_ = get_target_nn(nn_7)
nn_8_ = get_target_nn(nn_8)
nn_9_ = get_target_nn(nn_9)
nn_10_ = get_target_nn(nn_10)





In [ ]:
nn_2_[9]["Machine Learning"].intersection(nn_3_[9]["Machine Learning"])

In [ ]:
type(nn_10_[0]["Reinforcement Learning"])

In [ ]:
nn_10_

[{'markov': {'cross-entropy',
   'eigen-analysis',
   'high-dimensional',
   'hyperparameter-optimized',
   'hyperparameters',
   'log-likelihood',
   'low-dimensional',
   'multi-variate',
   'parameterss.',
   'trace-norm'}},
 {'deep learning': {'bonawitz',
   'datasets.',
   'ecom',
   'embeddings',
   'jean.lafond',
   'lafond',
   'olga.klopp',
   'parameterss.',
   'rouby',
   'soft-thresholding'}},
 {'supervised': {'conversely',
   'eigen-analysis',
   'hyper-parameter',
   'hyperparameters',
   'inferential',
   'matrix-vector',
   'parameterss.',
   'real-valued',
   'trace-norm',
   'word-vectors'}},
 {'gradient descent': {'combinatorial',
   'cross-entropy',
   'eigen-analysis',
   'log-likelihood',
   'multi-variate',
   'non-linearity',
   'parameterss.',
   'real-valued',
   'stochastic',
   'trace-norm'}},
 {'supervised learning': {'conversely',
   'eigen-analysis',
   'hyper-parameter',
   'hyperparameters',
   'inferential',
   'matrix-vector',
   'parameterss.',
   'r

In [ ]:
all_slice_nn = [nn_2_, nn_3_, nn_4_ , nn_5_ , nn_6_, nn_7_ , nn_8_ , nn_9_ , nn_10_]

In [ ]:
import math

def log_stability(A, B, keyword):
  a = len(A.intersection(B))
  b = len(A - B)
  # print(f"{keyword} : Intersection {a} - Subtract {b} :")
  if a == 0:
    return round( math.log10(1 / 10 ), 3)
  elif b == 0:
    return  -1 * round( math.log10( 10  ) , 3)
  else:
    return round ( math.log10( a  /  b  ) , 3)


def calc_stability(nn, keyword):
    i = 0
    stability = []
    for _,n in enumerate(nn):
       i = _ + 1

       if i < len(nn):
          stability.append( log_stability(nn[_], nn[i] , keyword) )

    return stability

def extract_keyword_nns(target_nn_10tw, keyword):
    keyword_neighbors_all_windows = []
    for _ , target_nn in enumerate(target_nn_10tw):

      keyword_neighbors_all_windows.extend([target[keyword] for index, target in enumerate(target_nn) if list(target.keys())[0] == keyword ])
    
    s = calc_stability(keyword_neighbors_all_windows, keyword)
    
    return s

def extract_stability(target_nn_10tw):
    s_n = []
    for _ , keyword in enumerate(monitering_keywords):
       s_n.append( { keyword: extract_keyword_nns(target_nn_10tw, keyword) } )

    return s_n

In [ ]:
keyword_stab = extract_stability(all_slice_nn)
keyword_stab

[{'markov': [-1.0, -1.0, -1.0, -1.0, -0.954, -0.602, -0.602, -0.954]},
 {'deep learning': [-0.368, -0.368, -0.368, -0.602, -1.0, -1.0, -1.0, -1.0]},
 {'supervised': [-1.0, -1.0, -1.0, -0.954, -1.0, -0.954, -1.0, -1.0]},
 {'gradient descent': [-1.0, -0.954, -1.0, -1.0, -0.954, -1.0, -1.0, -0.954]},
 {'supervised learning': [-1.0, -1.0, -1.0, -1.0, -1.0, -0.954, -1.0, -0.954]},
 {'neural network': [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]},
 {'dimensionality reduction': [-0.954,
   -0.954,
   -0.954,
   -1.0,
   -1.0,
   -0.602,
   -0.368,
   -1.0]},
 {'component analysis': [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]},
 {'time series': [-1.0, -1.0, -1.0, -1.0, -0.954, -0.602, -1.0, -0.954]},
 {'neural': [-1.0, -1.0, -1.0, -1.0, -0.368, -1.0, -0.954, -1.0]},
 {'neighbor': [-1.0, -1.0, -1.0, -0.602, -1.0, -1.0, -1.0, -1.0]},
 {'monte carlo': [-1.0, -1.0, -0.954, -1.0, -1.0, -1.0, -0.954, -0.954]},
 {'reinforcement learning': [-1.0,
   -0.954,
   -1.0,
   -0.954,
   -1.0,
   -1.

In [ ]:
log_stability(nn_8_[0]["Reinforcement Learning"], nn_7_[0]["Reinforcement Learning"])


In [ ]:
v = [n for n in nn_2_] 



list(nn_2_[0].keys())[0]